d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Experiment Tracking

The machine learning life cycle involves training multiple algorithms using different hyperparameters and libraries, all with different performance results and trained models.  This lesson explores tracking those experiments to organize the machine learning life cycle.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Introduce tracking ML experiments in MLflow
 - Log an experiment and explore the results in the UI
 - Record parameters, metrics, and a model
 - Query past runs programatically

Over the course of the machine learning life cycle...<br><br>

* Data scientists test many different models 
* Using various libraries 
* Each with different hyperparameters

Tracking these various results poses an organizational challenge, including... <br><br>

* Storing experiments
* Results
* Models
* Supplementary artifacts
* Code
* Data snapshots

### Tracking Experiments with MLflow

MLflow Tracking is...<br><br>

* a logging API specific for machine learning 
* agnostic to libraries and environments that do the training
* organized around the concept of **runs**, which are executions of data science code
* runs are aggregated into **experiments** where many runs can be a part of a given experiment 
* An MLflow server can host many experiments.

Each run can record the following information:<br><br>

- **Parameters:** Key-value pairs of input parameters such as the number of trees in a random forest model
- **Metrics:** Evaluation metrics such as RMSE or Area Under the ROC Curve
- **Artifacts:** Arbitrary output files in any format.  This can include images, pickled models, and data files
- **Source:** The code that originally ran the experiment

Experiments can be tracked using libraries in Python, R, and Java as well as by using the CLI and REST calls

-sandbox
<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlflow-tracking.png" style="height: 400px; margin: 20px"/></div>

In [6]:
%run "./Includes/Classroom-Setup"

-sandbox
### Experiment Logging and UI

MLflow is an open source software project developed by Databricks available to developers regardless of which platform they are using.  Databricks hosts MLflow for you, which reduces deployment configuration and adds security benefits.  As of February 2019,  <a href="https://docs.databricks.com/release-notes/release-types.html" target="_blank">MLflow is in public preview</a> and is therefore accessible on all Databricks workspaces.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See <a href="https://mlflow.org/docs/latest/quickstart.html#" target="_blank">the MLflow quickstart guide</a> for details on setting up MLflow locally or on your own server.

-sandbox
MLflow has number of dependencies, which have been installed for you for this class.  If you are running this code on a different cluster from what has been provisioned for you, uncomment and run the following code to install this library:<br><br>

 - `mlflow==0.9.0`
 
<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Installing packages like this only works with the Databricks Runtime, not the ML Runtime.  If you are using the ML Runtime, <a href="https://docs.databricks.com/user-guide/libraries.html" target="_blank">install these packages using the Clusters UI instead.</a>

In [9]:
# dbutils.library.installPyPI("mlflow", "0.9.0")
# dbutils.library.restartPython()

Import a dataset of Airbnb listings and featurize the data.  We'll use this to train a model.

In [11]:
import pandas as pd

df = pd.read_csv("/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv")

Perform a train/test split.

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

**Navigate to the MLflow UI by clicking on the `Runs` button on the top of the screen.**

Log a basic experiment by doing the following:<br><br>

1. Start an experiment using `mlflow.start_run()` and passing it a name for the run
2. Train your model
3. Log the model using `mlflow.sklearn.log_model()`
4. Log the model error using `mlflow.log_metric()`
5. Print out the run id using `run.info.run_uuid`

In [16]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

with mlflow.start_run(run_name="Basic RF Experiment") as run:
  # Create model, train it, and create predictions
  rf = RandomForestRegressor()
  rf.fit(X_train, y_train)
  predictions = rf.predict(X_test)
  
  # Log model
  mlflow.sklearn.log_model(rf, "random-forest-model")
  
  # Create metrics
  mse = mean_squared_error(y_test, predictions)
  print("  mse: {}".format(mse))
  
  # Log metrics
  mlflow.log_metric("mse", mse)
  
  runID = run.info.run_uuid
  experimentID = run.info.experiment_id
  
  print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))

mse: 6352.189090909091
Inside MLflow Run with run_id 74fcbb7a5ff34c289160cf16f521725c and experiment_id 1121904905105799

-sandbox
Examine the results in the UI.  Look for the following:<br><br>

1. The `Experiment ID`
2. The artifact location.  This is where the artifacts are stored in DBFS, which is backed by the Azure blob store
3. The time the run was executed.  **Click this to see more information on the run.**
4. The code that executed the run.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlflow-ui-lesson2.png" style="height: 400px; margin: 20px"/></div>

-sandbox
After clicking on the time of the run, take a look at the following:<br><br>

1. The Run ID will match what we printed above
2. The model that we saved, included a picked version of the model as well as the Conda environment and the `MLmodel` file, which will be discussed in the next lesson.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlflow-ui-lesson2b.png" style="height: 400px; margin: 20px"/></div>

Now take a look at the directory structure backing this experiment.  This allows you to retrieve artifacts.

In [20]:
from mlflow.tracking import MlflowClient

artifactURL = MlflowClient().get_experiment(experimentID).artifact_location
dbutils.fs.ls(artifactURL)

Out[11]: 
[FileInfo(path='dbfs:/databricks/mlflow/1121904905105799/0c21167e67894a86b4dc825868d9c826/', name='0c21167e67894a86b4dc825868d9c826/', size=0),
 FileInfo(path='dbfs:/databricks/mlflow/1121904905105799/74fcbb7a5ff34c289160cf16f521725c/', name='74fcbb7a5ff34c289160cf16f521725c/', size=0),
 FileInfo(path='dbfs:/databricks/mlflow/1121904905105799/b6310ba8841f49a182d156b3a0598d90/', name='b6310ba8841f49a182d156b3a0598d90/', size=0)]

Take a look at the contents of `random-forest-model`, which match what we see in the UI.

In [22]:
modelURL = artifactURL + "/" + runID + "/artifacts/random-forest-model"
dbutils.fs.ls(modelURL)

Out[12]: 
[FileInfo(path='dbfs:/databricks/mlflow/1121904905105799/74fcbb7a5ff34c289160cf16f521725c/artifacts/random-forest-model/MLmodel', name='MLmodel', size=357),
 FileInfo(path='dbfs:/databricks/mlflow/1121904905105799/74fcbb7a5ff34c289160cf16f521725c/artifacts/random-forest-model/conda.yaml', name='conda.yaml', size=119),
 FileInfo(path='dbfs:/databricks/mlflow/1121904905105799/74fcbb7a5ff34c289160cf16f521725c/artifacts/random-forest-model/model.pkl', name='model.pkl', size=2529837)]

### Parameters, Metrics, and Artifacts

But wait, there's more!  In the last example, you logged the run name, an evaluation metric, and your model itself as an artifact.  Now let's log parameters, multiple metrics, and other artifacts including the feature importances.

First, create a function to perform this.

In [24]:
userhome


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1234437178347043> in <module> () 
 ----> 1 userhome

 NameError : name 'userhome' is not defined

In [25]:
def log_rf(experimentID, run_name, params, X_train, X_test, y_train, y_test):
  import os
  import matplotlib.pyplot as plt
  import mlflow.sklearn
  import seaborn as sns
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

  with mlflow.start_run(experiment_id=experimentID, run_name=run_name) as run:
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")

    # Log params
    [mlflow.log_param(param, value) for param, value in params.items()]

    # Create metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("  mse: {}".format(mse))
    print("  mae: {}".format(mae))
    print("  R2: {}".format(r2))

    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)  
    mlflow.log_metric("r2", r2)  
    
    # Create feature importance
    importance = pd.DataFrame(list(zip(df.columns, rf.feature_importances_)), 
                                columns=["Feature", "Importance"]
                              ).sort_values("Importance", ascending=False)
    outdir = userhome.replace("dbfs:", "/dbfs") + "/ml-production/"
    
    csvPath = outdir + "feature-importance.csv"
    importance.to_csv(csvPath, index=False)
    mlflow.log_artifact(csvPath)

    # Create and lot plot
    fig, ax = plt.subplots()

    sns.residplot(predictions, y_test, lowess=True)
    plt.xlabel("Predicted values for Price ($)")
    plt.ylabel("Residual")
    plt.title("Residual Plot")

    figPath =  outdir + "residuals.png" 
    fig.savefig(figPath)

    mlflow.log_artifact(figPath)

    display(fig)
    return run.info.run_uuid

Run with new parameters.

In [27]:
params = {
  "n_estimators": 100,
  "max_depth": 5,
  "random_state": 42
}

log_rf(experimentID, "Second Run", params, X_train, X_test, y_train, y_test)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1121904905105825> in <module> () 
 5 }
 6 
 ----> 7 log_rf ( experimentID , "Second Run" , params , X_train , X_test , y_train , y_test ) 

 <command-1121904905105823> in log_rf (experimentID, run_name, params, X_train, X_test, y_train, y_test) 
 36 columns = [ "Feature" , "Importance" ] 
 37 ).sort_values("Importance", ascending=False)
 ---> 38 outdir = userhome . replace ( "dbfs:" , "/dbfs" ) + "/ml-production/" 
 39 
 40 csvPath = outdir + "feature-importance.csv" 

 NameError : name 'userhome' is not defined

Check the UI to see how this appears.  Take a look at the artifact to see where the plot was saved.

Now, run a third run.

In [29]:
params_1000_trees = {
  "n_estimators": 1000,
  "max_depth": 10,
  "random_state": 42
}

log_rf(experimentID, "Third Run", params_1000_trees, X_train, X_test, y_train, y_test)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1121904905105827> in <module> () 
 5 }
 6 
 ----> 7 log_rf ( experimentID , "Third Run" , params_1000_trees , X_train , X_test , y_train , y_test ) 

 <command-1121904905105823> in log_rf (experimentID, run_name, params, X_train, X_test, y_train, y_test) 
 36 columns = [ "Feature" , "Importance" ] 
 37 ).sort_values("Importance", ascending=False)
 ---> 38 outdir = userhome . replace ( "dbfs:" , "/dbfs" ) + "/ml-production/" 
 39 
 40 csvPath = outdir + "feature-importance.csv" 

 NameError : name 'userhome' is not defined

-sandbox
### Querying Past Runs

You can query past runs programatically in order to use this data back in Python.  The pathway to doing this is an `MlflowClient` object. 

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You can also set tags for runs using `client.set_tag(run.info.run_uuid, "tag_key", "tag_value")`

In [31]:
from  mlflow.tracking import MlflowClient

client = MlflowClient()

Now list all the runs for your experiment using `.list_run_infos()`, which takes your `experiment_id` as a parameter.

In [33]:
client.list_run_infos(experimentID)

Pull out a few fields and create a Pandas DataFrame with it.

In [35]:
runs = pd.DataFrame([(run.run_uuid, run.start_time, run.artifact_uri) for run in client.list_run_infos(experimentID)])
runs.columns = ["run_uuid", "start_time", "artifact_uri"]

display(runs)

Pull the last run and take a look at the associated artifacts.

In [37]:
last_run = runs.sort_values("start_time", ascending=False).iloc[0]

dbutils.fs.ls(last_run["artifact_uri"]+"/random-forest-model/")

Return the evaluation metrics for the last run.

In [39]:
client.get_run(last_run.run_uuid).data.metrics

Reload the model and take a look at the feature importance.

In [41]:
import mlflow.sklearn

model = mlflow.sklearn.load_model(path="random-forest-model", run_id=last_run["run_uuid"]) # Path must match the argument given in the log_model command
model.feature_importances_

## Review
**Question:** What can MLflow Tracking log?  
**Answer:** MLflow can log the following:
- **Parameters:** inputs to a model
- **Metrics:** the performance of the model
- **Artifacts:** any object including data, models, and images
- **Source:** the original code, including the commit hash if linked to git

**Question:** How do you log experiments?  
**Answer:** Experiments are logged by first creating a run and using the logging methods on that run object (e.g. `run.log_param("MSE", .2)`).

**Question:** Where do logged artifacts get saved?  
**Answer:** Logged artifacts are saved in a directory of your choosing.  On Databricks, this would be DBFS, or the Databricks File System, which backed by a blob store.

**Question:** How can I query past runs?  
**Answer:** This can be done using an `MlflowClient` object.  This allows you do everything you can within the UI programatically so you never have to step outside of your programming environment.

## Next Steps

Start the next lesson, [Packaging ML Projects]($./03-Packaging-ML-Projects ).

## Additional Topics & Resources

**Q:** Where can I find the MLflow docs?  
**A:** <a href="https://www.mlflow.org/docs/latest/index.html" target="_blank">You can find the docs here.</a>

**Q:** What is a good general resource for machine learning?  
**A:** <a href="https://www-bcf.usc.edu/~gareth/ISL/" target="_blank">_An Introduction to Statistical Learning_</a> is a good starting point for the themes and basic approaches to machine learning.

**Q:** Where can I find out more information on machine learning with Spark?  
**A:** Check out the Databricks blog <a href="https://databricks.com/blog/category/engineering/machine-learning" target="_blank">dedicated to machine learning</a>

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>